# **Conformación del corpus de comentarios de redes sociales de eventos educativos**

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import pandas as pd
import os
import pickle
import re, string
import numpy as np

In [ ]:
path_file = '/gdrive/MyDrive/DSRP/Historico/Datos originales/df_UnidoLimpio.xlsx'
df = pd.read_excel(path_file)

In [ ]:
df.head(5)

,Comentario,clasificacion,Relevancia,Comentario_limpio
0,"Buenos días, Juana Nancy Gonza Calderon les sa...",No Considerar,0,buenos días juana nancy gonza calderon les sal...
1,Buenas tardes! soy de la Universidad Nacional ...,No Considerar,0,buenas tardes soy de la universidad nacional d...
2,"I.E ""Juan Aurich Pastor"" de Batangrande - Lamb...",No Considerar,0,ie juan aurich pastor de batangrande lambayeq...
3,ESTAN FALLECIENDO LOS DIRECTORES,No Considerar,0,estan falleciendo los directores
4,"Buenas tardes colegas, saludos desde Piura, do...",No Considerar,0,buenas tardes colegas saludos desde piura doce...


In [ ]:
df2 = df.drop_duplicates()

In [ ]:
def clean_text(text):
    patron_1 = re.compile(r'(?P<l1>([A-Z]))\.(?P<l2>([A-Z]))')
    text = re.sub(patron_1, '\g<l1>\g<l2>', text)
    text = text.lower()
    text = re.sub(r'[áàä]', 'a', text)
    text = re.sub(r'[éèë]', 'e', text)
    text = re.sub(r'[íìï]', 'i', text)
    text = re.sub(r'[óòö]', 'o', text)
    text = re.sub(r'[úùü]', 'u', text)
    patron_2 = re.compile(r'(?P<l3>([iesp]))\.(?P<l4>([espt]))')
    text = re.sub(patron_2, '\g<l3>\g<l4>', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub('[¿°]', '', text)
    text = re.sub(r'\[.*¿!¡?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[,-:]', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r' i e ', ' ie ', text)
    if re.search(r'.$', text)[0] == ' ':
      text = re.sub(r'.$', '', text)
    else:
      text = text
    if text[0] == ' ':
      text = text[1:]
    else:
      text = text
    text = re.sub(r'si+', 'si', text)
    return text

def replace_space_by_nan(text):
  if len(text) == 0:
    return np.nan
  else:
    return text

def modify_classification(text):
  if text == 'Considerar':
    return 'considerar'
  else:
    return 'no-considerar'

In [ ]:
df2['Comentario_procesado'] = df2['Comentario'].apply(lambda x: clean_text(x))

In [ ]:
df2.head(5)

,Comentario,clasificacion,Relevancia,Comentario_limpio,Comentario_procesado
0,"Buenos días, Juana Nancy Gonza Calderon les sa...",No Considerar,0,buenos días juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...
1,Buenas tardes! soy de la Universidad Nacional ...,No Considerar,0,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...
2,"I.E ""Juan Aurich Pastor"" de Batangrande - Lamb...",No Considerar,0,ie juan aurich pastor de batangrande lambayeq...,ie juan aurich pastor de batangrande lambayequ...
3,ESTAN FALLECIENDO LOS DIRECTORES,No Considerar,0,estan falleciendo los directores,estan falleciendo los directores
4,"Buenas tardes colegas, saludos desde Piura, do...",No Considerar,0,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...


In [ ]:
df2['Comentario_procesado2'] = df2['Comentario_procesado'].apply(lambda x: replace_space_by_nan(x))

In [ ]:
df2.head(5)

,Comentario,clasificacion,Relevancia,Comentario_limpio,Comentario_procesado,Comentario_procesado2
0,"Buenos días, Juana Nancy Gonza Calderon les sa...",No Considerar,0,buenos días juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...
1,Buenas tardes! soy de la Universidad Nacional ...,No Considerar,0,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...
2,"I.E ""Juan Aurich Pastor"" de Batangrande - Lamb...",No Considerar,0,ie juan aurich pastor de batangrande lambayeq...,ie juan aurich pastor de batangrande lambayequ...,ie juan aurich pastor de batangrande lambayequ...
3,ESTAN FALLECIENDO LOS DIRECTORES,No Considerar,0,estan falleciendo los directores,estan falleciendo los directores,estan falleciendo los directores
4,"Buenas tardes colegas, saludos desde Piura, do...",No Considerar,0,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...


In [ ]:
df3 = df2.dropna()

In [ ]:
df3.head(5)

,Comentario,clasificacion,Relevancia,Comentario_limpio,Comentario_procesado,Comentario_procesado2
0,"Buenos días, Juana Nancy Gonza Calderon les sa...",No Considerar,0,buenos días juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...
1,Buenas tardes! soy de la Universidad Nacional ...,No Considerar,0,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...
2,"I.E ""Juan Aurich Pastor"" de Batangrande - Lamb...",No Considerar,0,ie juan aurich pastor de batangrande lambayeq...,ie juan aurich pastor de batangrande lambayequ...,ie juan aurich pastor de batangrande lambayequ...
3,ESTAN FALLECIENDO LOS DIRECTORES,No Considerar,0,estan falleciendo los directores,estan falleciendo los directores,estan falleciendo los directores
4,"Buenas tardes colegas, saludos desde Piura, do...",No Considerar,0,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...


In [ ]:
df3['Clasificacion_modificado'] = df3['clasificacion'].apply(lambda x: modify_classification(x))

In [ ]:
df3.head(5)

,Comentario,clasificacion,Relevancia,Comentario_limpio,Comentario_procesado,Comentario_procesado2,Clasificacion_modificado
0,"Buenos días, Juana Nancy Gonza Calderon les sa...",No Considerar,0,buenos días juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...,buenos dias juana nancy gonza calderon les sal...,no-considerar
1,Buenas tardes! soy de la Universidad Nacional ...,No Considerar,0,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...,buenas tardes soy de la universidad nacional d...,no-considerar
2,"I.E ""Juan Aurich Pastor"" de Batangrande - Lamb...",No Considerar,0,ie juan aurich pastor de batangrande lambayeq...,ie juan aurich pastor de batangrande lambayequ...,ie juan aurich pastor de batangrande lambayequ...,no-considerar
3,ESTAN FALLECIENDO LOS DIRECTORES,No Considerar,0,estan falleciendo los directores,estan falleciendo los directores,estan falleciendo los directores,no-considerar
4,"Buenas tardes colegas, saludos desde Piura, do...",No Considerar,0,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...,buenas tardes colegas saludos desde piura doce...,no-considerar


In [ ]:
df3['Comentario_procesado2'][200]

'buenas noches profesor wilber como se guarda y compartir a los niños desde tarma'

In [ ]:
corpus_dict = {'sentences': list(df3.Comentario_procesado2), 'types': list(df3.Clasificacion_modificado)}

with open('/gdrive/MyDrive/DSRP/Data/corpus_comments_2021-09-23', 'wb') as corpus_d:
  pickle.dump(corpus_dict, corpus_d, protocol=pickle.HIGHEST_PROTOCOL)

## 1. Lectura y visualización del corpus

In [ ]:
import pickle

with open('/gdrive/MyDrive/DSRP/Data/corpus_comments_2021-09-23', 'rb') as d:
    txt_dict = pickle.load(d)

comments_data, type_data = txt_dict['sentences'], txt_dict['types']

In [ ]:
import pandas as pd

# Show some examples in table
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame({'comments': comments_data, 'type_': type_data})

df_small = pd.DataFrame(columns=['comments', 'type_'])
j = 0
for i in df.type_.unique():
  df_small.loc[j] = df[df.type_==i].iloc[0]
  j = j+1
  
df_small

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,comments,type_
0,buenos dias juana nancy gonza calderon les saluda desde lima metropolitana del iespp manuel gonzalez prada presente,no-considerar
1,prestar atencion y saber ponerse en el lugar de la otra y actuar de manera asertiva,considerar


In [ ]:
comments_data

['buenos dias juana nancy gonza calderon les saluda desde lima metropolitana del iespp manuel gonzalez prada presente',
 'buenas tardes soy de la universidad nacional del altip',
 'ie juan aurich pastor de batangrande lambayeque presente',
 'estan falleciendo los directores',
 'buenas tardes colegas saludos desde piura docente de ingles de la ie ricardo palma ugel la union',
 'esta muy interesante',
 'freddy callupe buenas tardes de la ie jorge chavez dartnell',
 'buenas tardes colegas de la ie la divina providencia',
 'muchas gracias profesor boris por la exelente ponencia bendiciones',
 'derecho que no cumplen el estado',
 'ya pero me daran qaliwarma',
 'piura huancabamba',
 'buenas tardes ie huata cusco',
 'fe y esperanza',
 'vivian kreis tuesta tello iespp picota',
 'buenas tarde karla quispe',
 'nos pueden compartir el ppt',
 'buenas noches profesores buenas noches desde huancavelica',
 'buenas tardes rosa pineda ie',
 'hola buenas tardes a todos los presentes soy la estudiante sh

In [ ]:
import re

# Obtención de las frecuencias
vocab_word = {}
for comm in comments_data:
  for word in comm.split():
    key = word.lower()
    vocab_word[key] = vocab_word.get(key, 0) + 1

vocab_word = {k: v for k, v in sorted(vocab_word.items(), key=lambda item: item[1])}

# Selección de las N palabras con mayor frecuencia
cnt1, cnt2, cnt3 = 0, 0, 0
for word, freq in vocab_word.items():
  if freq == 1:
    cnt1 += 1
  elif freq == 2:
    cnt2 += 1
  elif freq == 3:
    cnt3 += 1
print('Número de palabras con frecuencia de 1: ', cnt1)
print('Número de palabras con frecuencia de 2: ', cnt2)
print('Número de palabras con frecuencia de 3: ', cnt3)
N = int(0.02 * cnt1)
words2replace = list(vocab_word.keys())[:N]
for word in words2replace:
  if re.match(r'.+[ar|er|ir]', word):
    words2replace.remove(word)

# Reemplazo en todas las oraciones
pattern = r'\b' + r'\b|\b'.join(words2replace) + r'\b'
for i, comm in enumerate(comments_data):
  comments_data[i] = re.sub(pattern, 'unk', comm.lower())

Número de palabras con frecuencia de 1:  8683
Número de palabras con frecuencia de 2:  2277
Número de palabras con frecuencia de 3:  1183


In [ ]:
comments_data

['buenos dias juana nancy gonza calderon les saluda desde lima metropolitana del iespp manuel gonzalez prada presente',
 'buenas tardes soy de la universidad nacional del unk',
 'ie juan aurich pastor de batangrande lambayeque presente',
 'estan falleciendo los directores',
 'buenas tardes colegas saludos desde piura docente de ingles de la ie ricardo palma ugel la union',
 'esta muy interesante',
 'freddy callupe buenas tardes de la ie jorge chavez dartnell',
 'buenas tardes colegas de la ie la divina providencia',
 'muchas gracias profesor boris por la exelente ponencia bendiciones',
 'derecho que no cumplen el estado',
 'ya pero me daran qaliwarma',
 'piura huancabamba',
 'buenas tardes ie unk cusco',
 'fe y esperanza',
 'vivian unk tuesta tello iespp picota',
 'buenas tarde karla quispe',
 'nos pueden compartir el ppt',
 'buenas noches profesores buenas noches desde huancavelica',
 'buenas tardes rosa pineda ie',
 'hola buenas tardes a todos los presentes soy la estudiante sharon l

## 2. Tokenización

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

# Tokenización
word_tokenizer = Tokenizer(filters='\n')
#word_tokenizer = Tokenizer(filters='\n', oov_token='unk')
word_tokenizer.fit_on_texts(comments_data)
'''
w2i_dict = {}
i2w_dict = {}
for i in range(len(word_tokenizer.word_index)):
  w2i_dict[word_tokenizer.index_word[i + 2]] = i + 1
  i2w_dict[i + 1] = word_tokenizer.index_word[i + 2]

word_tokenizer.word_index = w2i_dict
word_tokenizer.index_word = i2w_dict
'''
word_index = word_tokenizer.word_index

type_tokenizer = Tokenizer(filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n')
type_tokenizer.fit_on_texts(type_data)
type_index = type_tokenizer.word_index

num_words = len(word_index)
num_types = len(type_index)

print('Tokenización completada!')
print('Words: {:4d}'.format(num_words))
print('Intents1: {:4d}'.format(num_types))

Tokenización completada!
Words: 16499
Intents1:    2


In [ ]:
type_index

{'considerar': 2, 'no-considerar': 1}

In [ ]:
word_index

{'de': 1,
 'la': 2,
 'que': 3,
 'el': 4,
 'y': 5,
 'en': 6,
 'buenas': 7,
 'a': 8,
 'no': 9,
 'los': 10,
 'tardes': 11,
 'con': 12,
 'es': 13,
 'para': 14,
 'ie': 15,
 'se': 16,
 'por': 17,
 'las': 18,
 'del': 19,
 'todos': 20,
 'estudiantes': 21,
 'como': 22,
 'gracias': 23,
 'un': 24,
 'muy': 25,
 'desde': 26,
 'ugel': 27,
 'una': 28,
 'lo': 29,
 'esta': 30,
 'soy': 31,
 'presente': 32,
 'mi': 33,
 'docente': 34,
 'si': 35,
 'hay': 36,
 'nos': 37,
 'colegas': 38,
 'pero': 39,
 'estudiante': 40,
 'al': 41,
 'saludos': 42,
 'su': 43,
 'trabajo': 44,
 'mas': 45,
 'san': 46,
 'muchas': 47,
 'educacion': 48,
 'interesante': 49,
 'docentes': 50,
 'sus': 51,
 'favor': 52,
 'me': 53,
 'internet': 54,
 'ya': 55,
 'aprendizaje': 56,
 'o': 57,
 'iespp': 58,
 'tienen': 59,
 'todo': 60,
 'cuando': 61,
 'ser': 62,
 'son': 63,
 'este': 64,
 'nuestros': 65,
 'porque': 66,
 'hacer': 67,
 'mis': 68,
 'puede': 69,
 'solo': 70,
 'maestros': 71,
 'practica': 72,
 'nuestra': 73,
 'bien': 74,
 'niños': 75,

## 3. Preparación de la data (generators)

In [ ]:
comments_data_CC = []; type_data_CC = []
comments_data_NC = []; type_data_NC = []

for i in range(len(comments_data)):
  if type_data[i]=='considerar':
    comments_data_CC.append(comments_data[i]); type_data_CC.append(type_data[i])
  else:
    comments_data_NC.append(comments_data[i]); type_data_NC.append(type_data[i])

In [ ]:
print(len(comments_data_CC), len(comments_data_NC))

10026 10030


In [ ]:
from random import shuffle
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

def split_train_val_test(comments_data, type_data):

  # Barajando los datos del corpus
  num_samples = len(comments_data)
  group_data = list(zip(comments_data, type_data))
  shuffle(group_data)
  comments_data, type_data = zip(*group_data)

  # Separación de los datos
  train_len = round(0.9 * num_samples)
  val_len = round(0.5 * (num_samples - train_len))
  # Entrenamiento - Validación - Prueba
  comments_train_data, comments_val_data, comments_test_data = comments_data[:train_len], comments_data[train_len:(train_len + val_len)], comments_data[(train_len + val_len):]
  type_train_data, type_val_data, type_test_data = type_data[:train_len], type_data[train_len:(train_len + val_len)], type_data[(train_len + val_len):]
  
  return comments_train_data, comments_val_data, comments_test_data, type_train_data, type_val_data, type_test_data

comments_data_CC_train, comments_data_CC_val, comments_data_CC_test, type_data_CC_train, type_data_CC_val, type_data_CC_test = split_train_val_test(comments_data_CC, type_data_CC)
comments_data_NC_train, comments_data_NC_val, comments_data_NC_test, type_data_NC_train, type_data_NC_val, type_data_NC_test = split_train_val_test(comments_data_NC, type_data_NC)

comments_train_data = comments_data_CC_train + comments_data_NC_train
comments_val_data = comments_data_CC_val + comments_data_NC_val
comments_test_data = comments_data_CC_test + comments_data_NC_test
type_train_data = type_data_CC_train + type_data_NC_train
type_val_data = type_data_CC_val + type_data_NC_val
type_test_data = type_data_CC_test + type_data_NC_test

In [ ]:
corpus = dict()
corpus['comments_t'] = comments_train_data
corpus['comments_v'] = comments_val_data
corpus['comments_te'] = comments_test_data
corpus['type_t'] = type_train_data
corpus['type_v'] = type_val_data
corpus['type_te'] = type_test_data

with open('/gdrive/MyDrive/DSRP/Data/corpus_dc', 'wb') as c_dc:
  pickle.dump(corpus, c_dc, protocol=pickle.HIGHEST_PROTOCOL)

print('Number of training samples: %s' % len(comments_train_data))
print('Number of validation samples: %s' % len(comments_val_data))
print('Number of testing samples: %s' % len(comments_test_data))

Number of training samples: 18050
Number of validation samples: 1004
Number of testing samples: 1002


In [ ]:
with open('/gdrive/MyDrive/DSRP/Data/word_tokenizer', 'wb') as w_tok:
  pickle.dump(word_tokenizer, w_tok, protocol=pickle.HIGHEST_PROTOCOL)

with open('/gdrive/MyDrive/DSRP/Data/type_tokenizer', 'wb') as t_tok:
  pickle.dump(type_tokenizer, t_tok, protocol=pickle.HIGHEST_PROTOCOL)